In [2]:
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

from pyvirtualdisplay import Display

virtual_display = Display(visible = 0, size=(1400, 900))
virtual_display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0

In [3]:
!pip install gym[box2d]
!pip install stable-baselines3[extra]
!pip install huggingface_sb3
!pip install pyglet
!pip install ale-py==0.7.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 448 kB 7.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 9.4 MB/s 
     |████████████████████████████████| 1.5 MB 62.4 MB/s 
     |████████████████████████████████| 1.6 MB 47.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=4feb6355eda2365a9148693eccc663a14671608342354989aa68f55b6dca4eab
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=b008701cd6923762f27e7f9d71f2a1e3f6e1cc5f0af37b47317d469be18b1418
 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 8.5 MB/s 
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.7.5
    Uninstalling ale-py-0.7.5:
      Successfully uninstalled ale-py-0.7.5


In [5]:
import gym

from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import notebook_login

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

In [6]:
import gym

env = gym.make("LunarLander-v2")

observation = env.reset()

for _ in range(20):
  action = env.action_space.sample()
  print("Action taken:", action)

  observation, reward, done, info = env.step(action)

  if done:
    print("Environment is rest")
    observation = env.reset()

Action taken: 1
Action taken: 3
Action taken: 2
Action taken: 3
Action taken: 0
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 0
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 3
Action taken: 2
Action taken: 0
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 2


In [17]:
env = gym.make("LunarLander-v2")
env.reset()
print("____OBSERVATION SPACE____ \n")
print("Observation Space shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())

____OBSERVATION SPACE____ 

Observation Space shape (8,)
Sample observation [-0.50239503 -0.08143948 -0.18179484 -1.0715425  -1.1669674  -0.42138952
  0.49216974 -1.6693641 ]


In [18]:
print("____ACTION SPACE____ \n")
print("Action Space shape", env.action_space.n)
print("Action Space sample", env.action_space.sample())

____ACTION SPACE____ 

Action Space shape 4
Action Space sample 1


In [19]:
env = make_vec_env('LunarLander-v2', n_envs=16)

In [20]:
model = PPO(policy='MlpPolicy',
            env = env,
            n_steps=2048,
            batch_size=64,
            n_epochs=8,
            gamma=0.98,
            gae_lambda=0.98,
            ent_coef=0.01,
            verbose=1)

Using cuda device


In [21]:
model.learn(total_timesteps=1000000)
model_name="ppo-LunarLander-v2"
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 89.1     |
|    ep_rew_mean     | -181     |
| time/              |          |
|    fps             | 4343     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 32768    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 89          |
|    ep_rew_mean          | -140        |
| time/                   |             |
|    fps                  | 1979        |
|    iterations           | 2           |
|    time_elapsed         | 33          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.009817342 |
|    clip_fraction        | 0.0793      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.000214    |
|    learning_rate        | 0.

In [22]:
eval_env = gym.make("LunarLander-v2")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward=156.35 +/- 112.6167249620409


In [14]:
notebook_login()
!git config --global credential.helper store

Login successful
Your token has been saved to /root/.huggingface/token


In [23]:
import gym

from stable_baselines3 import ppo
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

env_id = "LunarLander-v2"

model_architecture = "PPO"

repo_id=""

commit_message = "Increase time and change parameters"

eval_env = DummyVecEnv([lambda: gym.make(env_id)])

package_to_hub(model=model,
               model_name=model_name,
               model_architecture=model_architecture,
               env_id=env_id,
               eval_env=eval_env,
               repo_id=repo_id,
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue and use
push_to_hub instead.


/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:82: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.8. Pass `repo_id` instead.
  FutureWarning,
/content/hub/LunarLander-v2 is already a clone of https://huggingface.co/jianyang/LunarLander-v2. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Saving video to /content/-step-0-to-step-1000.mp4
ℹ Pushing repo LunarLander-v2 to the Hugging Face Hub


Upload file replay.mp4:   2%|1         | 3.34k/193k [00:00<?, ?B/s]

Upload file ppo-LunarLander-v2/policy.pth:   8%|7         | 3.34k/42.2k [00:00<?, ?B/s]

Upload file ppo-LunarLander-v2/policy.optimizer.pth:   4%|4         | 3.34k/82.9k [00:00<?, ?B/s]

Upload file ppo-LunarLander-v2.zip:   2%|2         | 3.34k/141k [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/jianyang/LunarLander-v2
   9180485..2611b6f  main -> main



ℹ Your model is pushed to the hub. You can view your model here:
https://huggingface.co/jianyang/LunarLander-v2


'https://huggingface.co/jianyang/LunarLander-v2'